## Baggage Distance Detection

**Note**: I have also attached a full fledged video ```baggage.mkv``` and sample ```baggage_test.mkv``` so that you can also see if the results are relevant for that.<br>
**P.S**: For practical case, please change the average pixel per meter according to the distance measured as it is not a fixed metric and will change according to the camera position.<br> In the ```baggage.mkv``` video, the camera position changes frequently which will give inaccurate results. Make sure that this model is impplemented on a stationary camera

**P.S.1**: The Current model gives us around 3-4fps. You could speed this up by using Intel's <i>OpenVino environment</i> which will almost double the fps rate. This environment can be exported to small systems as well(eg: Raspberry Pi)

## Importing the Dependencies

In [2]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import os
import cv2
import math
import itertools
import datetime

## Loading the model

Note: If model loading is showing error in anyway, just install the ```requirement.txt``` inside the yolov5 folder. If still showing error after that, then restart the kernel and try again

In [4]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\sreenivasulu/.cache\torch\hub\master.zip


requirements: matplotlib>=3.2.2 not found and is required by YOLOv5, attempting auto-update...
requirements: Command 'pip install 'matplotlib>=3.2.2'' returned non-zero exit status 1.
requirements: Pillow>=7.1.2 not found and is required by YOLOv5, attempting auto-update...
requirements: Command 'pip install 'Pillow>=7.1.2'' returned non-zero exit status 1.
requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...
requirements: Command 'pip install 'PyYAML>=5.3.1'' returned non-zero exit status 1.
requirements: tqdm>=4.41.0 not found and is required by YOLOv5, attempting auto-update...
requirements: Command 'pip install 'tqdm>=4.41.0'' returned non-zero exit status 1.
requirements: seaborn>=0.11.0 not found and is required by YOLOv5, attempting auto-update...
requirements: Command 'pip install 'seaborn>=0.11.0'' returned non-zero exit status 1.


YOLOv5  2021-10-5 torch 1.9.1+cpu CPU



Fusing layers... 
Model Summary: 224 layers, 7266973 parameters, 0 gradients
Adding AutoShape... 


In [5]:
model

AutoShape(
  (model): Model(
    (model): Sequential(
      (0): Focus(
        (conv): Conv(
          (conv): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (2): C3(
        (cv1): Conv(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv3): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (m): Sequential(
          (0): Bottleneck(
            (cv1): Conv(
              (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
              (act): SiLU(inplace=True)
            )
    

In [6]:
img = 'https://ultralytics.com/images/zidane.jpg'

In [7]:
results = model(img)
results.print()

image 1/1: 720x1280 2 persons, 2 ties
Speed: 4160.5ms pre-process, 2601.2ms inference, 393.2ms NMS per image at shape (1, 3, 384, 640)


In [8]:
#Displaying results
cv2.imshow('Detection', np.squeeze(results.render()))
cv2.waitKey(0)
cv2.destroyAllWindows()

### Checking GPU Instance

In [9]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


### Functions for Inference of Single Image

In [11]:
# Calculates coordinates from the result dataframe

def calc_coord(df):
    temp_coord = []
    for i in range(len(df)):
        temp_coord.append(df.iloc[i, :4].tolist())
    return temp_coord

# Calculates centroids from the resulting coordinates

def calc_centroid(coord):
    temp_cent = []
    for i in coord:
        temp_cent.append([int(i[0] + (i[2] - i[0])/2), int(i[1] + (i[3] - i[1])/2)])
    return temp_cent

# Calculates permutations of persons to suitcases

def calc_permutations(l1, l2):
    tot_permutations = []
    for r in itertools.product(l1,l2):
        tot_permutations.append([r[0], r[1]])
    return tot_permutations

# Calculates distance from each resulting centroid permutation

def calc_distance(cent):
    dist = math.sqrt((cent[1][0] - cent[0][0])**2 + (cent[1][1] - cent[0][1])**2)
    return dist

# Updates the dataframe to include another column for easy mapping of claimed baggages

def update_df(df3):
    df3['id'] = 0
    for i in range(len(df3)):
        df3.loc[i, 'id']= i+1
    return df3

# Maps the centroids to the id column so as to identify claimed baggages

def map_cent2id(suit_list, df):
    cent_id = {}
    near_person = {}
    for i in range(len(df)):
        cent_id[str(suit_list[i])] = df['id'][i]
        near_person[str(suit_list[i])] = False
    return cent_id, near_person

In [12]:
# Calculates the inference implementing all the functions above for a single image or single frame of video
def img_inference(frame):
    
    frame = cv2.resize(frame, (0,0), fx=0.5, fy=0.5)
    
    #Pixel per meter (ImageWidth/Field of view in meters)- Change this parameter when changing to a stationary camera
    average_ppm = frame.shape[1]/5
    
    res = model(frame)
    dict1 = res.pandas().xyxy[0]
    for i in range(len(dict1)):
        x1, y1, x2, y2 = int(np.floor(dict1['xmin'][i])), int(np.floor(dict1['ymin'][i])), int(np.floor(dict1['xmax'][i])), int(np.floor(dict1['ymax'][i]))
        if dict1.loc[i, 'name'] == 'person':
            frame = cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
        elif dict1.loc[i, 'name'] == 'suitcase':
            frame = cv2.rectangle(frame, (x1,y1), (x2,y2), (128,0,145), 2)
    person_df = dict1[dict1['name'] == 'person']
    person_df = person_df.reset_index().drop(columns=['index'])
    suitcase_df = dict1[dict1['name'] == 'suitcase']
    suitcase_df = suitcase_df.reset_index().drop(columns=['index'])
    person_coord = calc_coord(person_df)
    suitcase_coord = calc_coord(suitcase_df)
    person_cent = calc_centroid(person_coord)
    suitcase_cent = calc_centroid(suitcase_coord)
    person_df = update_df(person_df.copy())
    suitcase_df = update_df(suitcase_df.copy())
    perms = calc_permutations(person_cent, suitcase_cent)
    mapped, person_near = map_cent2id(suitcase_cent, suitcase_df)
    for perm in perms:
        distance = calc_distance(perm)
        distance_meter = distance/average_ppm

        if distance_meter < 1.2:
#             print('claimed', distance_meter)
            person_near[str(perm[1])] = True
        else:
            if person_near[str(perm[1])] == False:
                id1 = mapped[str(perm[1])]
                unattended_label = suitcase_df.iloc[id1-1,:2].tolist()
                unattended_label = tuple([int(np.floor(x-10)) for x in unattended_label])
                frame = cv2.putText(frame,"Unattended Baggage" ,unattended_label, cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA, False)
#                 print('Unattended Baggage', distance_meter)
    return frame

## Testing on an Image

In [13]:
image = cv2.imread('test2.png')
r = model(image)
r.print()

image 1/1: 1080x1920 2 persons, 1 car, 1 backpack, 2 suitcases
Speed: 15.2ms pre-process, 585.1ms inference, 0.0ms NMS per image at shape (1, 3, 384, 640)


In [14]:
# Inferencing Image to find the distance between person and suitcase
img1 = img_inference(image)
cv2.imshow("Image", img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Video Feed Testing

**Note:** Press 'q' to exit the video feed

In [16]:
#cap = cv2.VideoCapture('baggage_test.mkv')
cap = cv2.VideoCapture('The CCTV People Demo 2_360P.mp4')
if cap.isOpened() == False:
    print('Error')
fps_start_time = datetime.datetime.now()
fps = 0
total_frames = 0
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        frame = img_inference(frame)
        total_frames +=1
        fps_end_time = datetime.datetime.now()
        time_diff = fps_end_time - fps_start_time
        if time_diff.seconds == 0:
            fps = 0
        else:
            fps = (total_frames/time_diff.seconds)
        
        fps_text = "FPS: {}".format(fps)
        frame = cv2.putText(frame, fps_text, (5,30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (125, 0, 225), 1)
        cv2.imshow('Video Testing', frame)
        
    else:
        break
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()